# Pasture and Grazing Management: Planning Rotations

This notebook helps you plan grazing rotations, calculate carrying capacity,
track pasture recovery, and build a seasonal forage budget. All examples use
a small sheep operation so you can adapt the numbers to your own farm.

## How to Run This Notebook

**Requirements:** Python 3.12+ and project dependencies installed.

1. Install dependencies (includes pandas, numpy, matplotlib):
   ```
   uv sync --extra data
   ```
2. Start Jupyter:
   ```
   uv run jupyter lab
   ```
   (or `uv run jupyter notebook`)
3. Open this notebook and run cells from top to bottom by pressing **Shift+Enter** on each cell.
4. Replace the sample data with your own pasture measurements to get personalized results.

**Note:** If a cell produces an error, make sure you have run all the cells above it first.

## Setup

Import the libraries we need and configure display settings.

In [ ]:
# Standard library
from datetime import datetime, timedelta

# Data and math
import pandas as pd

try:
    import matplotlib.pyplot as plt
    import numpy as np

    HAS_MATPLOTLIB = True
    try:
        plt.style.use("seaborn-v0_8-whitegrid")
    except OSError:
        try:
            plt.style.use("ggplot")
        except OSError:
            pass  # Fall back to default style
except ImportError:
    HAS_MATPLOTLIB = False
    print("Optional: pip install matplotlib for charts")

try:
    from tabulate import tabulate

    HAS_TABULATE = True
except ImportError:
    HAS_TABULATE = False
    print("Optional: pip install tabulate for prettier tables")

# Configuration
pd.set_option("display.max_columns", None)

## Carrying Capacity Calculation

Carrying capacity tells you how many animals your land can support without
degrading the pasture. We measure it in **Animal Unit Months (AUMs)**.

Key terms:
- **Animal Unit (AU)**: One 1,000-lb cow with calf. A sheep is about 0.2 AU.
- **AUM**: The forage one AU eats in a month (~800 lbs dry matter).
- **Utilization rate**: The percentage of forage you plan to graze (never 100%).

We will use an 80-acre sample farm producing 3,000 lbs of dry matter per acre
with a 55% utilization rate.

In [ ]:
# Farm parameters
total_acres = 80
forage_yield_lbs_per_acre = 3000  # lbs dry matter per acre per season
utilization_rate = 0.55           # graze 55%, leave 45% for recovery
au_monthly_intake_lbs = 800       # lbs DM one AU eats per month
sheep_au = 0.2                    # one sheep = 0.2 AU

# Calculate available forage
total_forage_lbs = total_acres * forage_yield_lbs_per_acre
available_forage_lbs = total_forage_lbs * utilization_rate

# AUMs the farm can support
total_aums = available_forage_lbs / au_monthly_intake_lbs

# Stocking rate for a 6-month grazing season
grazing_season_months = 6
max_au = total_aums / grazing_season_months
max_sheep = max_au / sheep_au

print(f"Total forage produced:  {total_forage_lbs:,.0f} lbs DM")
print(f"Available after 55%:   {available_forage_lbs:,.0f} lbs DM")
print(f"Total AUMs:            {total_aums:,.1f}")
print(f"Max AU for {grazing_season_months}-mo season: {max_au:,.1f} AU")
print(f"Max sheep:             {max_sheep:,.0f} head")

## Paddock Rotation Planner

Divide the farm into paddocks and plan how long animals stay in each one.
Good rotational grazing gives each paddock enough rest days for regrowth.

In [ ]:
# Define paddocks
raw_paddocks = pd.DataFrame({
    "paddock_name": ["North Hill", "Creek Bottom", "South Flat",
                     "East Ridge", "West Meadow"],
    "acres": [18, 14, 20, 12, 16],
    "current_forage_height_in": [8, 10, 6, 9, 7],
    "min_rest_days": [30, 25, 35, 30, 28],
})

print("Paddock inventory:")
if HAS_TABULATE:
    print(tabulate(raw_paddocks, headers="keys", tablefmt="grid",
                   showindex=False))
else:
    print(raw_paddocks.to_string())

### Building a Rotation Schedule

We assign grazing days proportional to each paddock's acreage and
generate a calendar showing when animals move.

In [ ]:
# Total grazing days in a rotation cycle
total_rotation_days = raw_paddocks["min_rest_days"].max() + len(raw_paddocks)

# Grazing days per paddock proportional to acreage
raw_paddocks["graze_days"] = (
    (raw_paddocks["acres"] / raw_paddocks["acres"].sum())
    * total_rotation_days
).round().astype(int)

# Build the rotation calendar
start_date = datetime(2026, 4, 15)
rotation_schedule = []
current_date = start_date

for _, row in raw_paddocks.iterrows():
    entry = current_date
    exit_date = entry + timedelta(days=int(row["graze_days"]))
    rotation_schedule.append({
        "paddock_name": row["paddock_name"],
        "entry_date": entry.strftime("%Y-%m-%d"),
        "exit_date": exit_date.strftime("%Y-%m-%d"),
        "graze_days": row["graze_days"],
    })
    current_date = exit_date

schedule_by_paddock = pd.DataFrame(rotation_schedule)
print("Rotation schedule (first cycle):")
if HAS_TABULATE:
    print(tabulate(schedule_by_paddock, headers="keys",
                   tablefmt="grid", showindex=False))
else:
    print(schedule_by_paddock.to_string())

## Recording Grazing Events

Every time you move animals, log the event. This record helps you see
which paddocks are being over- or under-utilized over time.

In [ ]:
# Sample grazing log
raw_grazing_log = pd.DataFrame({
    "paddock": ["North Hill", "Creek Bottom", "South Flat",
                "East Ridge", "West Meadow"],
    "entry_date": pd.to_datetime([
        "2026-04-15", "2026-04-24", "2026-05-04",
        "2026-05-18", "2026-05-27",
    ]),
    "exit_date": pd.to_datetime([
        "2026-04-24", "2026-05-03", "2026-05-17",
        "2026-05-27", "2026-06-07",
    ]),
    "animal_count": [55, 55, 55, 55, 55],
    "forage_in_inches": [8, 10, 6, 9, 7],
    "forage_out_inches": [3, 4, 2.5, 3.5, 3],
})

# Calculate utilization percentage
raw_grazing_log["utilization_pct"] = (
    (raw_grazing_log["forage_in_inches"]
     - raw_grazing_log["forage_out_inches"])
    / raw_grazing_log["forage_in_inches"] * 100
).round(1)

clean_grazing_log = raw_grazing_log.copy()
print("Grazing event log:")
if HAS_TABULATE:
    print(tabulate(clean_grazing_log, headers="keys",
                   tablefmt="grid", showindex=False))
else:
    print(clean_grazing_log.to_string())

## Pasture Recovery Monitoring

After animals leave a paddock, measure forage height weekly to track
regrowth. Healthy pastures recover in a characteristic S-curve.
The chart below shows simulated recovery for each paddock.

In [ ]:
# Simulate regrowth curves (logistic growth model)
if HAS_MATPLOTLIB:
    days_after_grazing = np.arange(0, 46)
    paddock_names = ["North Hill", "Creek Bottom", "South Flat",
                     "East Ridge", "West Meadow"]
    residual_heights = [3.0, 4.0, 2.5, 3.5, 3.0]
    target_heights = [8.0, 10.0, 6.0, 9.0, 7.0]

    fig, ax = plt.subplots(figsize=(10, 5))

    for name, residual, target in zip(
        paddock_names, residual_heights, target_heights, strict=False
    ):
        carrying_capacity = target
        growth_rate = 0.12
        recovery = residual + (carrying_capacity - residual) / (
            1 + np.exp(-growth_rate * (days_after_grazing - 20))
        )
        ax.plot(days_after_grazing, recovery, linewidth=2, label=name)

    ax.set_xlabel("Days After Grazing")
    ax.set_ylabel("Forage Height (inches)")
    ax.set_title("Pasture Recovery Curves by Paddock")
    ax.legend(loc="lower right")
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart")

## Seasonal Forage Budget

A forage budget compares monthly supply (pasture growth) against demand
(what your animals eat). Months where demand exceeds supply are deficit
months -- you will need hay, stockpiled forage, or reduced stocking.

In [ ]:
# Monthly forage production (lbs DM) -- typical cool-season grass pattern
months = ["Apr", "May", "Jun", "Jul", "Aug", "Sep"]
monthly_growth_pct = [0.10, 0.25, 0.20, 0.12, 0.13, 0.20]

forage_supply_by_month = pd.DataFrame({
    "month": months,
    "supply_lbs": [pct * available_forage_lbs for pct in monthly_growth_pct],
})

# Monthly demand: 55 sheep * 0.2 AU * 800 lbs/AU/month
sheep_count = 55
monthly_demand_lbs = sheep_count * sheep_au * au_monthly_intake_lbs
forage_supply_by_month["demand_lbs"] = monthly_demand_lbs

# Surplus or deficit
forage_supply_by_month["balance_lbs"] = (
    forage_supply_by_month["supply_lbs"]
    - forage_supply_by_month["demand_lbs"]
)

print("Monthly forage budget:")
if HAS_TABULATE:
    print(tabulate(forage_supply_by_month, headers="keys",
                   tablefmt="grid", showindex=False, floatfmt=",.0f"))
else:
    print(forage_supply_by_month.to_string())

In [ ]:
# Bar chart: supply vs demand by month
if HAS_MATPLOTLIB:
    x = np.arange(len(months))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 5))
    bars_supply = ax.bar(x - width / 2,
                         forage_supply_by_month["supply_lbs"],
                         width, label="Supply", color="#4CAF50")
    bars_demand = ax.bar(x + width / 2,
                         forage_supply_by_month["demand_lbs"],
                         width, label="Demand", color="#FF7043")

    ax.set_xlabel("Month")
    ax.set_ylabel("Forage (lbs DM)")
    ax.set_title("Monthly Forage Supply vs. Demand")
    ax.set_xticks(x)
    ax.set_xticklabels(months)
    ax.legend()

    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart")

# Highlight deficit months
deficit_months = forage_supply_by_month[
    forage_supply_by_month["balance_lbs"] < 0
]["month"].tolist()

if deficit_months:
    print(f"Deficit months: {', '.join(deficit_months)}")
    print("Consider hay supplementation or reducing stock.")
else:
    print("No deficit months -- forage supply meets demand.")

## Connecting Grazing to Ecological Outcomes

Good grazing management is the foundation of land health. The metrics in
this notebook -- utilization rate, rest days, recovery curves -- connect
directly to the ecological health indicators tracked in the
[Ecological Monitoring (EOV)](ecological-monitoring-eov.ipynb) notebook.

When you record grazing events here, you create the data you need to
show land improvement over time. This is especially valuable for:

- **EQIP and CSP applications**: Demonstrating planned grazing systems
- **Savory Institute EOV assessments**: Linking management to outcomes
- **Adaptive management**: Adjusting stocking rates based on real data

See the EOV notebook for site-level soil health, biodiversity, and
water cycle assessments that complement this grazing data.

## Conclusion

This notebook covered the core tools for managing pasture and grazing:

- [x] **Carrying capacity**: Calculated AUMs and maximum stocking rate
- [x] **Rotation planning**: Built a paddock schedule with rest periods
- [x] **Grazing records**: Logged moves with forage utilization
- [x] **Recovery monitoring**: Visualized regrowth curves
- [x] **Forage budgeting**: Compared monthly supply vs. demand
- [x] **Ecological connection**: Linked grazing data to land health

### Next Steps

1. Replace sample data with your own pasture measurements
2. Adjust the AU equivalents if you run cattle or mixed species
3. Update forage yields based on your soil tests and rainfall
4. Explore the [EOV notebook](ecological-monitoring-eov.ipynb) for
   ecological monitoring that pairs with this grazing data